# 페이스북 크롤링

In [31]:
# 페이지 스크롤 횟수 설정
num_scrolldown = 20

In [32]:
from bs4 import BeautifulSoup as bs
import time
import random
from datetime import datetime
from selenium import webdriver
import chromedriver_autoinstaller as ca

In [33]:
import pandas as pd

## 크롤링할 사이트 접속

In [34]:
# 크롤링할 웹사이트 주소
url = 'https://www.facebook.com/GyeonggiKorea.en'

# webdriver 로드
ca.install(True)
driver = webdriver.Chrome()
driver.get(url)
time.sleep(3)
driver.implicitly_wait(5)

# 창 최대화
driver.maximize_window()
time.sleep(1)

## 페이지 스크롤 다운

In [35]:
# 페이지 스크롤 다운
for i in range(num_scrolldown):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep( random.uniform(1,3) )

## 크롤링

In [37]:
# 페이지 크롤링
soup = bs(driver.page_source, 'lxml')
print(soup.title.text)

Gyeonggi Korea | Facebook


In [7]:
# chromedriver 종료
driver.quit()

# 게시글 크롤링

## 게시글 수집

In [38]:
# 게시글 개수
div = soup.find_all('div', 'du4w35lb l9j0dhe7')
print("수집한 게시글 수 : ", len(div) )

수집한 게시글 수 :  67


## 날짜 수집

In [41]:
# 글 날짜 수집
num_date_list = []
for one in div:
    raw_date_info = one.find_all('div', 'qzhwtbm6 knvmm38d')[1]
    # raw_date = raw_date_info.find('a', 'oajrlxb2 g5ia77u1 qu0x051f esr5mh6w e9989ue4 r7d6kgcz rq0escxv nhd2j8a9 nc684nl6 p7hjln8o kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x jb3vyjys rz4wbd8a qt6c0cv9 a8nywdso i1ao9s8h esuyzwwr f1sip0of lzcic4wl gmql0nx0 gpro0wi8 b1v8xokw').attrs['aria-label']
    raw_date = raw_date_info.find('a').attrs['aria-label']

    # # 날짜 형식 변환해서 리스트로 저장
    if ('시간' in raw_date) or ('분' in raw_date):
        temp = datetime.today().strftime("%Y-%m-%d")
        num_date_list.append(temp)
    elif '어제' in raw_date:
        temp = datetime.today().strftime("%Y-%m-%d")
        temp = temp.split('-')
        year = temp[0]
        month = temp[1]
        day = int( temp[2] ) - 1
        temp_date = f'{year}-{month:0>2}-{day:0>2}'
        num_date_list.append(temp_date)
    else:
        temp = raw_date.split('일')[0]
        temp = temp.replace(' ', '')
        temp = temp.replace('년', '-')
        temp = temp.replace('월', '-')
        temp_li = temp.split('-')
        if len(temp_li) == 3 :
            year = temp_li[0]
            month = temp_li[1]
            day = temp_li[2]
            temp_date = f'{year}-{month:0>2}-{day:0>2}'
            num_date_list.append(temp_date)
        elif len(temp_li) == 2:
            year = datetime.today().strftime("%Y")
            month = temp_li[0]
            day = temp_li[1]
            temp_date = f'{year}-{month:0>2}-{day:0>2}'
            num_date_list.append(temp_date)
        else:
            temp_day = int(temp_li[0]) + 1
            temp = datetime.today().strftime("%Y-%m-%d")
            temp = temp.split('-')
            year = temp[0]
            month = temp[1]
            day = int( temp[2] ) - temp_day
            temp_date = f'{year}-{month:0>2}-{day:0>2}'
            num_date_list.append(temp_date)
            

print("변환한 날짜 리스트 원소수 : ", len(num_date_list) )

변환한 날짜 리스트 원소수 :  67


In [42]:
num_date_list[:5]

['2022-04-29', '2022-04-27', '2022-04-27', '2022-04-25', '2022-04-25']

## 제목 수집

In [43]:
# 글 제목 수집
title_list = []
for one in div:
    raw_title = one.find('div', 'kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x c1et5uql ii04i59q')
    if raw_title == None:
        raw_title = one.find('div', 'kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x c1et5uql')
    title = raw_title.text
    title_list.append(title)

print("수집한 제목 수 : ", len(title_list))

수집한 제목 수 :  67


In [44]:
title_list[:5]

['Enjoy spring at the Everland Tulip Festival in Gyeonggi Province!',
 'Gyeonggi Expands Non-Contact ‘Tagless’ Bus Fare Payment Service Throughout Province ',
 'Song of Gyeonggi-do (‘Rest in Gyeonggi-do’)Lyrics: Kim Jihun / Composed by: Oh-Nong Project',
 'Gyeonggi Korea X 마마무 Mamamoo Gyeonggi-do (ReMiFa) Solar (TiDo)',
 'Travel via ‘Jebudo Cable Car’ in Gyeonggi’s Hwaseong City']

## 조회수 수집

In [45]:
# 조회수
num_view_list = []
for one in div:
    num_view_list.append(0)

print("수집한 조회수 수 : ", len(num_view_list))

수집한 조회수 수 :  67


In [46]:
num_view_list[:5]

[0, 0, 0, 0, 0]

# 좋아요 수 수집

In [47]:
# 좋아요 수 수집
num_like_list = []
for one in div:
    raw_lsc_info = one.find('div', 'bp9cbjyn j83agx80 buofh1pr ni8dbmo4 stjgntxs')
    if raw_lsc_info == None:
        num_like = 0
        num_best = 0
    else:
        # 좋아요 수 수집
        raw_like_info = raw_lsc_info.find('span', 'np69z8it et4y5ytx j7g94pet b74d5cxt qw6c0r16 kb8x4rkr ed597pkb omcyoz59 goun2846 ccm00jje s44p3ltw mk2mc5f4 qxh1up0x qtyiw8t4 tpcyxxvw k0bpgpbk hm271qws rl04r1d5 l9j0dhe7 ov9facns kavbgo14')
        if raw_like_info == None:
            num_like = 0
        else:
            raw_like = raw_like_info.find('div', 'oajrlxb2 gs1a9yip g5ia77u1 mtkw9kbi tlpljxtp qensuy8j ppp5ayq2 goun2846 ccm00jje s44p3ltw mk2mc5f4 rt8b4zig n8ej3o3l agehan2d sk4xxmp2 rq0escxv nhd2j8a9 mg4g778l pfnyh3mw p7hjln8o kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x tgvbjcpo hpfvmrgz jb3vyjys rz4wbd8a qt6c0cv9 a8nywdso l9j0dhe7 i1ao9s8h esuyzwwr f1sip0of du4w35lb n00je7tq arfg74bv qs9ysxi8 k77z8yql pq6dq46d btwxx1t3 abiwlrkh p8dawk7l lzcic4wl').attrs['aria-label']
            if '천' in raw_like:
                num_like = raw_like
            else:
                num_like = int(raw_like[:-1].split(' ')[-1])
        # 최고에요 수 수집
        raw_best_info = raw_lsc_info.find('span', 'np69z8it et4y5ytx j7g94pet b74d5cxt qw6c0r16 kb8x4rkr ed597pkb omcyoz59 goun2846 ccm00jje s44p3ltw mk2mc5f4 qxh1up0x qtyiw8t4 tpcyxxvw k0bpgpbk hm271qws rl04r1d5 l9j0dhe7 ov9facns tkr6xdv7')
        if raw_best_info == None:
            num_best = 0
        else:
            raw_best = raw_best_info.find('div', 'oajrlxb2 gs1a9yip g5ia77u1 mtkw9kbi tlpljxtp qensuy8j ppp5ayq2 goun2846 ccm00jje s44p3ltw mk2mc5f4 rt8b4zig n8ej3o3l agehan2d sk4xxmp2 rq0escxv nhd2j8a9 mg4g778l pfnyh3mw p7hjln8o kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x tgvbjcpo hpfvmrgz jb3vyjys rz4wbd8a qt6c0cv9 a8nywdso l9j0dhe7 i1ao9s8h esuyzwwr f1sip0of du4w35lb n00je7tq arfg74bv qs9ysxi8 k77z8yql pq6dq46d btwxx1t3 abiwlrkh p8dawk7l lzcic4wl').attrs['aria-label']
            if '천' in raw_best:
                num_best = raw_best
            else:
                num_best = int(raw_best[:-1].split(' ')[-1])

    # 최종 좋아요 수
    if type(num_like) == str and type(num_best) == int:
        like = num_like + ' + ' + str(num_best)
        num_like_list.append(like)
    elif type(num_like) == int and type(num_best) == str:
        like = str(num_like) + ' + ' + num_best
        num_like_list.append(like)
    else:
        like = num_like + num_best
        num_like_list.append(like)

print('좋아요 수 리스트 원소수 : ',len(num_like_list))

좋아요 수 리스트 원소수 :  67


In [48]:
num_like_list[:5]

[92, 21, 14, 304, 36]

## 공유, 댓글 수 수집

In [49]:
# 공유 수, 댓글 수 수집
num_share_list = []
num_comment_list = []
for one in div:
    sc_info = one.find('div', 'bp9cbjyn j83agx80 pfnyh3mw p1ueia1e')
    if sc_info == None:
        num_share = 0
        num_comment = 0
    else:
        sc_list = sc_info.find_all('span', 'd2edcug0 hpfvmrgz qv66sw1b c1et5uql lr9zc1uh a8c37x1j fe6kdd0r mau55g9w c8b282yb keod5gw0 nxhoafnm aigsh9s9 d3f4x2em iv3no6db jq4qci2q a3bd9o3v b1v8xokw m9osqain')
        if (sc_list == None) or ( len(sc_list) == 0 ):
            num_share = 0
            num_comment = 0
        elif len(sc_list) == 1:
            for two in sc_list:
                two_str = two.text
                if '공유' in two_str:
                    try:
                        num_share = int(two_str[:-1].split(' ')[-1])
                        num_comment = 0
                    except:
                        num_share = two_str
                        num_comment = 0
                elif '댓글' in two_str:
                    try:
                        num_comment = int(two_str[:-1].split(' ')[-1])
                        num_share = 0
                    except:
                        num_comment = two_str
                        num_share = 0
        elif len(sc_list) == 2:
            for two in sc_list:
                two_str = two.text
                if '공유' in two_str:
                    try:
                        num_share = int(two_str[:-1].split(' ')[-1])
                    except:
                        num_share = two_str
                elif '댓글' in two_str:
                    try:
                        num_comment = int(two_str[:-1].split(' ')[-1])
                    except:
                        num_comment = two_str

    num_share_list.append(num_share)
    num_comment_list.append(num_comment)

print("공유 수의 개수 : ", len(num_share_list))
print("댓글 수의 개수 : ", len(num_comment_list))

공유 수의 개수 :  67
댓글 수의 개수 :  67


In [50]:
num_share_list[:5]

[9, 7, 1, 0, 4]

In [51]:
num_comment_list[:5]

[0, 0, 0, 0, 0]

### 로그인 안했을 때

In [52]:
# 게시글 링크 수집1(로그인❌)
link_list = []
for one in div:
    raw_link_info = one.find_all('div', 'qzhwtbm6 knvmm38d')[1]
    link = raw_link_info.find('a', 'oajrlxb2 g5ia77u1 qu0x051f esr5mh6w e9989ue4 r7d6kgcz rq0escxv nhd2j8a9 nc684nl6 p7hjln8o kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x jb3vyjys rz4wbd8a qt6c0cv9 a8nywdso i1ao9s8h esuyzwwr f1sip0of lzcic4wl gmql0nx0 gpro0wi8 b1v8xokw').attrs['href'].split('?')[0]
    link_list.append(link)

print('링크 리스트 원소수 : ',len(link_list))

링크 리스트 원소수 :  67


In [53]:
link_list[:5]

['https://www.facebook.com/GyeonggiKorea.en/posts/364891402333206',
 'https://www.facebook.com/GyeonggiKorea.en/posts/363733182449028',
 'https://www.facebook.com/GyeonggiKorea.en/posts/363607955794884',
 'https://www.facebook.com/GyeonggiKorea.en/posts/362458182576528',
 'https://www.facebook.com/GyeonggiKorea.en/posts/360680352754311']

## 수집한 데이터 확인

In [54]:
print("수집한 게시글 수 : ", len(div) )
print("변환한 날짜 리스트 원소수 : ", len(num_date_list) )
print("수집한 제목 수 : ", len(title_list))
print("수집한 조회수 수 : ", len(num_view_list))
print('좋아요 수 리스트 원소수 : ',len(num_like_list))
print("공유 수의 개수 : ", len(num_share_list))
print("댓글 수의 개수 : ", len(num_comment_list))
print('링크 리스트 원소수 : ',len(link_list))

수집한 게시글 수 :  67
변환한 날짜 리스트 원소수 :  67
수집한 제목 수 :  67
수집한 조회수 수 :  67
좋아요 수 리스트 원소수 :  67
공유 수의 개수 :  67
댓글 수의 개수 :  67
링크 리스트 원소수 :  67


## 데이터 셋 만들기

In [55]:
# 수집 데이터를 데이터 셋으로 만들기
df = {'날짜':num_date_list, '제목':title_list, '조회수':num_view_list, '좋아요_수':num_like_list, '공유수':num_share_list, '댓글수':num_comment_list, '링크':link_list}
data = pd.DataFrame(df)
data

,날짜,제목,조회수,좋아요_수,공유수,댓글수,링크
0,2022-04-29,Enjoy spring at the Everland Tulip Festival in...,0,92,9,0,https://www.facebook.com/GyeonggiKorea.en/post...
1,2022-04-27,Gyeonggi Expands Non-Contact ‘Tagless’ Bus Far...,0,21,7,0,https://www.facebook.com/GyeonggiKorea.en/post...
2,2022-04-27,Song of Gyeonggi-do (‘Rest in Gyeonggi-do’)Lyr...,0,14,1,0,https://www.facebook.com/GyeonggiKorea.en/post...
3,2022-04-25,Gyeonggi Korea X 마마무 Mamamoo Gyeonggi-do (ReMi...,0,304,0,0,https://www.facebook.com/GyeonggiKorea.en/post...
4,2022-04-25,Travel via ‘Jebudo Cable Car’ in Gyeonggi’s Hw...,0,36,4,0,https://www.facebook.com/GyeonggiKorea.en/post...
...,...,...,...,...,...,...,...
62,2022-01-01,Happy New Year! We hope you have a happy year...,0,808,20,34,https://www.facebook.com/GyeonggiKorea.en/post...
63,2021-12-31,"Good bye, 2021 Explore the top four destinatio...",0,1031,29,14,https://www.facebook.com/GyeonggiKorea.en/post...
64,2021-12-29,Let's Warm Up at Hot Springs in Gyeonggi Prov...,0,좋아요: 1천명 + 97,19,6,https://www.facebook.com/GyeonggiKorea.en/post...
65,2021-12-24,Romantic Winter Travel Destinations for Christ...,0,좋아요: 1천명 + 111,31,10,https://www.facebook.com/GyeonggiKorea.en/post...


## 데이터 셋. 날짜로 슬라이싱

In [56]:
data2 = data[data['날짜'] >= '2022-01-01']
data2

,날짜,제목,조회수,좋아요_수,공유수,댓글수,링크
0,2022-04-29,Enjoy spring at the Everland Tulip Festival in...,0,92,9,0,https://www.facebook.com/GyeonggiKorea.en/post...
1,2022-04-27,Gyeonggi Expands Non-Contact ‘Tagless’ Bus Far...,0,21,7,0,https://www.facebook.com/GyeonggiKorea.en/post...
2,2022-04-27,Song of Gyeonggi-do (‘Rest in Gyeonggi-do’)Lyr...,0,14,1,0,https://www.facebook.com/GyeonggiKorea.en/post...
3,2022-04-25,Gyeonggi Korea X 마마무 Mamamoo Gyeonggi-do (ReMi...,0,304,0,0,https://www.facebook.com/GyeonggiKorea.en/post...
4,2022-04-25,Travel via ‘Jebudo Cable Car’ in Gyeonggi’s Hw...,0,36,4,0,https://www.facebook.com/GyeonggiKorea.en/post...
...,...,...,...,...,...,...,...
58,2022-01-08,Catch this week's #GTV English #News for the l...,0,24,0,0,https://www.facebook.com/GyeonggiKorea.en/post...
59,2022-01-07,Contactless payment service launched on Gyeong...,0,67,1,1,https://www.facebook.com/GyeonggiKorea.en/post...
60,2022-01-05,How should you take care of dry skin in the wi...,0,352,20,0,https://www.facebook.com/GyeonggiKorea.en/post...
61,2022-01-02,Catch this week's #GTV English #News for the l...,0,34,0,0,https://www.facebook.com/GyeonggiKorea.en/post...


## 필요한 데이터만 추출

In [57]:
# 작성 게시글만 추출
removed_words = ["Catch this", "["]

# 제거할 단어 들어있는 행 인덱스 찾기
temp = data2['제목'].to_list()
remove_index = []
for i, s in enumerate(temp):
    for one in removed_words:
        if one in s:
            remove_index.append(i)
len(remove_index), remove_index

(15, [6, 10, 14, 17, 22, 26, 28, 31, 35, 38, 41, 44, 54, 58, 61])

In [58]:
# 중복 인덱스 제거
mod_remove_index = []
for i in remove_index:
    if i not in mod_remove_index:
        mod_remove_index.append(i)

len(mod_remove_index), mod_remove_index

(15, [6, 10, 14, 17, 22, 26, 28, 31, 35, 38, 41, 44, 54, 58, 61])

## 데이터 셋. 인덱스 내림차순으로 정렬

In [59]:
# 인덱스 리셋, 내림차순 정렬
data3 = data2.drop(mod_remove_index).reset_index(drop=True)
final_data = data3.sort_index(ascending=False)
final_data

,날짜,제목,조회수,좋아요_수,공유수,댓글수,링크
47,2022-01-01,Happy New Year! We hope you have a happy year...,0,808,20,34,https://www.facebook.com/GyeonggiKorea.en/post...
46,2022-01-05,How should you take care of dry skin in the wi...,0,352,20,0,https://www.facebook.com/GyeonggiKorea.en/post...
45,2022-01-07,Contactless payment service launched on Gyeong...,0,67,1,1,https://www.facebook.com/GyeonggiKorea.en/post...
44,2022-01-08,Gyeonggi-do’s Pyeongtaek is a certified child-...,0,768,8,12,https://www.facebook.com/GyeonggiKorea.en/post...
43,2022-01-13,"5 Metaverse Travel Spots in Siheung City, Gye...",0,548,12,7,https://www.facebook.com/GyeonggiKorea.en/post...
42,2022-01-15,Gyeonggi Province’s Anyang City runs Korea’s f...,0,584,16,6,https://www.facebook.com/GyeonggiKorea.en/post...
41,2022-01-17,Gyeonggi Province publishes K-culture report,0,433,15,8,https://www.facebook.com/GyeonggiKorea.en/post...
40,2022-01-20,Abandoned wastewater treatment plant transform...,0,73,3,3,https://www.facebook.com/GyeonggiKorea.en/post...
39,2022-01-24,Gyeonggi Peace Plaza: Filled with captivating ...,0,47,0,0,https://www.facebook.com/GyeonggiKorea.en/post...
38,2022-01-26,Gyeonggi Happiness Shop achieves record-break...,0,45,0,0,https://www.facebook.com/GyeonggiKorea.en/post...


## 엑셀 파일로 출력

In [60]:
# 수집한 데이터 파일료 출력
file_name = str(datetime.today().strftime("%Y%m%d-%H%M%S"))
file_name = 'facebook_' + file_name + '.xlsx'
path = 'crawled_data/facebook/'
final_data.to_excel(path+file_name, index=False, encoding='utf-8')

print(f"{file_name} 파일 생성 완료")

facebook_20220502-094505.xlsx 파일 생성 완료


# 코드 마지막